In [28]:
import numpy as np
import random
import itertools
import copy

In [29]:
goalkeepers = ['etsuko']
defenders = ['jazzie', 'helen', 'caroline']
centrals = ['vicky', 'olivia', 'keah', 'jaz']
wingers = ['keah', 'vicky', 'olivia', 'sian','jaz']
forwards = ['keah', 'olivia', 'vicky', 'shafa']

all_players = np.unique(goalkeepers + defenders + centrals + wingers + forwards).tolist()

permanent_gk = 0
if len(goalkeepers)==1:
    players_to_rotate = np.unique(defenders + centrals + wingers + forwards).tolist()
    permanent_gk = 1
else:
    players_to_rotate = all_players
    
n_players_to_rotate = len(players_to_rotate)

In [30]:
# player dictionary
player_dict = dict()
for player in players_to_rotate:
    positions = []
    if player in defenders:
        positions.append('D')
    if player in centrals:
        positions.append('C')
    if player in wingers:
        positions.append('W')
    if player in forwards:
        positions.append('F')
    player_dict[player] = positions
    
print(player_dict)

position_dict = {'D':defenders, 'C':centrals, 'W':wingers, 'F':forwards}
print(position_dict)

{'caroline': ['D'], 'helen': ['D'], 'jaz': ['C', 'W'], 'jazzie': ['D'], 'keah': ['C', 'W', 'F'], 'olivia': ['C', 'W', 'F'], 'shafa': ['F'], 'sian': ['W'], 'vicky': ['C', 'W', 'F']}
{'D': ['jazzie', 'helen', 'caroline'], 'C': ['vicky', 'olivia', 'keah', 'jaz'], 'W': ['keah', 'vicky', 'olivia', 'sian', 'jaz'], 'F': ['keah', 'olivia', 'vicky', 'shafa']}


In [31]:
# formation
n_defender = 2
n_central = 1
n_winger = 2
n_forward = 1

master_formation = ['D', 'D', 'C', 'W', 'W', 'F']
master_formation.sort()

In [32]:
# dictionary of sub periods
off_times = {1:[1,7], 2:[2,6], 3:[2,8]}
n_subs = n_players_to_rotate - 6

n_periods = 7
if n_subs==2:
    n_periods = 4
elif n_subs==3:
    n_periods = 3

In [33]:
n_players_to_rotate, n_subs, n_periods

(9, 3, 3)

In [34]:
def n_positions_on_pitch(player_list, position_list):
    n_players_on_pitch = 0
    for player in position_list:
        if player in player_list:
            n_players_on_pitch += 1
            
    return n_players_on_pitch

def check_valid_players(player_list):
    defenders_valid = True if n_positions_on_pitch(player_list, defenders)>=n_defender else False
    centrals_valid = True if n_positions_on_pitch(player_list, centrals)>=n_central else False
    wingers_valid = True if n_positions_on_pitch(player_list, wingers)>=n_winger else False
    forwards_valid = True if n_positions_on_pitch(player_list, forwards)>=n_forward else False
    
    return defenders_valid and centrals_valid and wingers_valid and forwards_valid

def check_valid_players_2(player_list):
    
    formation = {'D':[], 'C':[], 'W':[], 'F':[]}
    
    multi_position_players = []
    available_positions = []
    
    _formation = []

    
    for player in player_list:
        positions = player_dict[player]
        if len(positions)>1:
            multi_position_players.append(player)
            #available_positions.append(positions)
            available_positions.append(tuple(positions))
        else:
            position = positions[0]
            _formation.append(position)
            formation[position].append(player)
            
    
    multi_options = list(itertools.product(*available_positions))
    
    found_good_formation = False
    
    formations = []
    
    for m in multi_options:
        this_formation = copy.deepcopy(formation)
        
        new_formation = list(m) + _formation
        new_formation.sort()
        
        if new_formation == master_formation:
            found_good_formation = True
            for i in range(len(m)):
                this_formation[m[i]].append(multi_position_players[i])
            this_formation['S'] = list(set(players_to_rotate)-set(player_list))
            formations.append(this_formation)

    #for f in formations:
    #    print(f)
    
    if found_good_formation:
        return True, formations
    
    else:
        return False, []
    
def formation_output(formation):
    out_dict = {'Defenders':[], 'Central mid':[], 'Wingers':[], 'Forward':[], 'Subs':[]}
    out_dict['Defenders'] = formation['D']
    out_dict['Central mid'] = formation['C']
    out_dict['Wingers'] = formation['W']
    out_dict['Forward'] = formation['F']
    out_dict['Subs'] = formation['S']
    return out_dict
    

In [37]:
sub_combos = itertools.combinations(players_to_rotate, n_subs)
valid_combos = []
for combo in sub_combos:
    okay, formations = check_valid_players_2(list(set(players_to_rotate)-set(combo)))
    if okay:
        #for f in formation:
        #    print(f'formation: {f}')
        valid_combos.append(combo)
        
final_combos = []
#print(valid_combos)
blah = itertools.combinations(valid_combos, n_periods)
for b in blah:
    #print(b)
    if len(np.unique(list(sum(b,())))) == n_players_to_rotate:
        #print(b)
        final_combos.append(b)
        
choice = random.choice(final_combos)
final_subs = random.sample(choice, k=len(choice))

for sub in final_subs:
    print(formation_output(random.choice(check_valid_players_2(list(set(players_to_rotate)-set(sub)))[1])))

{'Defenders': ['jazzie', 'caroline'], 'Central mid': ['jaz'], 'Wingers': ['sian', 'vicky'], 'Forward': ['shafa'], 'Subs': ['helen', 'olivia', 'keah']}
{'Defenders': ['helen', 'caroline'], 'Central mid': ['olivia'], 'Wingers': ['vicky', 'keah'], 'Forward': ['shafa'], 'Subs': ['jaz', 'jazzie', 'sian']}
{'Defenders': ['helen', 'jazzie'], 'Central mid': ['keah'], 'Wingers': ['sian', 'jaz'], 'Forward': ['olivia'], 'Subs': ['vicky', 'caroline', 'shafa']}
